# 도구 호출 (Tool Call) 을 활용하면 GPT의 능력을 확장할 수 있다. 

In [1]:
%pip install langchain
%pip install langchain_openai

  Using cached langchain_text_splitters-0.3.8-py3-none-any.whl.metadata (1.9 kB)
  Using cached sqlalchemy-2.0.40-cp312-cp312-macosx_11_0_arm64.whl.metadata (9.6 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached zstandard-0.23.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (3.0 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.2 MB/s eta 0:00:00
Using cached langchain_text_splitters-0.3.8-py3-none-any.whl (32 kB)
Using cached PyYAML-6.0.2-cp312-cp312-macosx_11_0_arm64.whl (173 kB)
Using cached sqlalchemy-2.0.40-cp312-cp312-macosx_11_0_arm64.whl (2.1 MB)
Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl (54 kB)
Using cached zstandard-0.23.0-cp312-cp312

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
llm = ChatOpenAI(model="gpt-4o-mini")

llm.invoke([HumanMessage("잘 지냈어? 지금 몇시야?")])

AIMessage(content='저는 AI이기 때문에 감정은 없지만, 당신과 대화하는 것은 항상 기쁩니다! 현재 시각은 사용자에게 맞춰 알 수 없기 때문에, 시간은 직접 확인해 주셔야 합니다. 다른 도움이 필요하시면 언제든지 말씀해 주세요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 17, 'total_tokens': 77, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BWmT44ap84D0xKt7nPv1v1rqA9Hys', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--7655dec2-5625-45ad-b2af-3bd71cb822c3-0', usage_metadata={'input_tokens': 17, 'output_tokens': 60, 'total_tokens': 77, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

- 파이썬에서 컴퓨터의 현재 시각을 볼 수 있게 해주자

In [3]:
from datetime import datetime

print(datetime.now())

2025-05-14 01:09:21.123160


In [6]:
from langchain_core.tools import tool


@tool
def get_current_time() -> str:
    """현재 시각을 반환하는 함수 """
    return datetime.now().strftime("%H:%M:%S")

tools = [get_current_time, ]
tool_dict = {tool.name: tool for tool in tools}

print(tool_dict)


{'get_current_time': StructuredTool(name='get_current_time', description='현재 시각을 반환하는 함수', args_schema=<class 'langchain_core.utils.pydantic.get_current_time'>, func=<function get_current_time at 0x10c3f9d00>)}


In [7]:
llm_with_tools = llm.bind_tools(tools)
response = llm_with_tools.invoke([HumanMessage("잘 지냈어? 지금 몇시야?")])
print(response)

content='' additional_kwargs={'tool_calls': [{'id': 'call_zbLu41JB08DmDUNVRuO1K9GO', 'function': {'arguments': '{}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 50, 'total_tokens': 62, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BWmWjw9bYobxpKpaolRYAiekIRdKt', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--51c1452a-9a24-45d6-95b4-3e82d57816c0-0' tool_calls=[{'name': 'get_current_time', 'args': {}, 'id': 'call_zbLu41JB08DmDUNVRuO1K9GO', 'type': 'tool_call'}] usage_metadata={'input_tokens': 50, 'output_tokens': 12, 'total_tokens': 62, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_tok

In [9]:
from langchain_core.messages import SystemMessage

# (4) 사용자의 질문과 tools 사용하여 llm 답변 생성
messages = [
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    HumanMessage("부산은 지금 몇시야?"),
]

# (5) llm_with_tools를 사용하여 사용자의 질문에 대한 llm 답변 생성
response = llm_with_tools.invoke(messages)
messages.append(response)

# (6) 생성된 llm 답변 출력
for m in messages:
    m.pretty_print()

================================ System Message ================================

너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.
================================ Human Message =================================

부산은 지금 몇시야?
================================== Ai Message ==================================
Tool Calls:
  get_current_time (call_U9G9CtrYhqCZgeAzUJ6yQbb8)
 Call ID: call_U9G9CtrYhqCZgeAzUJ6yQbb8
  Args:


In [10]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]] # (7) tool_dict를 사용하여 도구 함수를 선택
    print(tool_call["args"]) # (8) 도구 호출 시 전달된 인자 출력
    tool_msg = selected_tool.invoke(tool_call) # (9) 도구 함수를 호출하여 결과를 반환
    messages.append(tool_msg)

for m in messages:
    m.pretty_print()

{}
================================ System Message ================================

너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.
================================ Human Message =================================

부산은 지금 몇시야?
================================== Ai Message ==================================
Tool Calls:
  get_current_time (call_U9G9CtrYhqCZgeAzUJ6yQbb8)
 Call ID: call_U9G9CtrYhqCZgeAzUJ6yQbb8
  Args:
================================= Tool Message =================================
Name: get_current_time

01:13:17


In [11]:
response = llm_with_tools.invoke(messages)

messages.append(response)

for m in messages:
    m.pretty_print()

================================ System Message ================================

너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.
================================ Human Message =================================

부산은 지금 몇시야?
================================== Ai Message ==================================
Tool Calls:
  get_current_time (call_U9G9CtrYhqCZgeAzUJ6yQbb8)
 Call ID: call_U9G9CtrYhqCZgeAzUJ6yQbb8
  Args:
================================= Tool Message =================================
Name: get_current_time

01:13:17
================================== Ai Message ==================================

현재 부산의 시각은 01:13입니다.


## 도구를 더 많이 쥐어주자. 
- 주식에 대한 이야기도 할 수 있게 하자

In [15]:
%pip install yfinance
%pip install tabulate


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [16]:
# yfinance 라이브러리를 이용하면 주식 정보를 쉽게 가져올 수 있다.

import yfinance as yf

yf.Ticker("AAPL").info

{'address1': 'One Apple Park Way',
 'city': 'Cupertino',
 'state': 'CA',
 'zip': '95014',
 'country': 'United States',
 'phone': '(408) 996-1010',
 'website': 'https://www.apple.com',
 'industry': 'Consumer Electronics',
 'industryKey': 'consumer-electronics',
 'industryDisp': 'Consumer Electronics',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and p

In [22]:
stock = yf.Ticker("AAPL")
history = stock.history(period="1mo", interval="1d")
history_md = history.to_markdown()

print(history_md)

| Date                      |    Open |    High |     Low |   Close |      Volume |   Dividends |   Stock Splits |
|:--------------------------|--------:|--------:|--------:|--------:|------------:|------------:|---------------:|
| 2025-04-14 00:00:00-04:00 | 211.163 | 212.661 | 200.897 | 202.255 | 1.01353e+08 |        0    |              0 |
| 2025-04-15 00:00:00-04:00 | 201.596 | 203.243 | 199.538 | 201.875 | 5.13439e+07 |        0    |              0 |
| 2025-04-16 00:00:00-04:00 | 198.1   | 200.437 | 192.118 | 194.016 | 5.97324e+07 |        0    |              0 |
| 2025-04-17 00:00:00-04:00 | 196.942 | 198.57  | 194.165 | 196.722 | 5.13343e+07 |        0    |              0 |
| 2025-04-21 00:00:00-04:00 | 193.017 | 193.546 | 189.561 | 192.907 | 4.67425e+07 |        0    |              0 |
| 2025-04-22 00:00:00-04:00 | 195.863 | 201.326 | 195.713 | 199.478 | 5.29764e+07 |        0    |              0 |
| 2025-04-23 00:00:00-04:00 | 205.73  | 207.728 | 202.534 | 204.332 | 5.29292e+0

## Pydantic
- 내가 원하는 형태로 정의할 수 있다. 

In [23]:
from pydantic import BaseModel, Field

class StockHistoryInput(BaseModel):
    ticker: str = Field(..., title="주식 코드", description="주식 코드 (예: AAPL)")
    period: str = Field(..., title="기간", description="주식 데이터 조회 기간 (예: 1d, 1mo, 1y)")

In [24]:


@tool
def get_yf_stock_history(stock_history_input: StockHistoryInput) -> str:
    """ 주식 종목의 가격 데이터를 조회하는 함수"""
    stock = yf.Ticker(stock_history_input.ticker)
    history = stock.history(period=stock_history_input.period)
    history_md = history.to_markdown() 

    return history_md

tools = [get_current_time, get_yf_stock_history]
tool_dict = {tool.name: tool for tool in tools}

llm_with_tools = llm.bind_tools(tools)

In [25]:
messages.append(HumanMessage("테슬라는 한달 전에 비해 주가가 올랐나 내렸나?"))

response = llm_with_tools.invoke(messages)
print(response)
messages.append(response)

for m in messages:
    m.pretty_print()

content='' additional_kwargs={'tool_calls': [{'id': 'call_KcZw4j69FLVuVgK7e97ZOhkF', 'function': {'arguments': '{"stock_history_input":{"ticker":"TSLA","period":"1mo"}}', 'name': 'get_yf_stock_history'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 197, 'total_tokens': 225, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BWmfEL8X9QZbVo1bZ32S3C3YoNpZP', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--521253ca-6469-49e7-a2ab-ee3eba620e53-0' tool_calls=[{'name': 'get_yf_stock_history', 'args': {'stock_history_input': {'ticker': 'TSLA', 'period': '1mo'}}, 'id': 'call_KcZw4j69FLVuVgK7e97ZOhkF', 'type': 'tool_call'}] usage_metadata={'inp

In [26]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]]
    print(tool_call["args"])
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    print(tool_msg)

{'stock_history_input': {'ticker': 'TSLA', 'period': '1mo'}}
content='| Date                      |    Open |   High |    Low |   Close |      Volume |   Dividends |   Stock Splits |\n|:--------------------------|--------:|-------:|-------:|--------:|------------:|------------:|---------------:|\n| 2025-04-14 00:00:00-04:00 | 258.36  | 261.8  | 245.93 |  252.35 | 1.00135e+08 |           0 |              0 |\n| 2025-04-15 00:00:00-04:00 | 249.91  | 258.75 | 247.54 |  254.11 | 7.95943e+07 |           0 |              0 |\n| 2025-04-16 00:00:00-04:00 | 247.61  | 251.97 | 233.89 |  241.55 | 1.12379e+08 |           0 |              0 |\n| 2025-04-17 00:00:00-04:00 | 243.47  | 244.34 | 237.68 |  241.37 | 8.34048e+07 |           0 |              0 |\n| 2025-04-21 00:00:00-04:00 | 230.26  | 232.21 | 222.79 |  227.5  | 9.7768e+07  |           0 |              0 |\n| 2025-04-22 00:00:00-04:00 | 230.96  | 242.79 | 229.85 |  237.97 | 1.20858e+08 |           0 |              0 |\n| 2025-04-23 00:00

In [27]:
response = llm_with_tools.invoke(messages)
messages.append(response)

for m in messages:
    m.pretty_print()

================================ System Message ================================

너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.
================================ Human Message =================================

부산은 지금 몇시야?
================================== Ai Message ==================================
Tool Calls:
  get_current_time (call_U9G9CtrYhqCZgeAzUJ6yQbb8)
 Call ID: call_U9G9CtrYhqCZgeAzUJ6yQbb8
  Args:
================================= Tool Message =================================
Name: get_current_time

01:13:17
================================== Ai Message ==================================

현재 부산의 시각은 01:13입니다.
================================ Human Message =================================

테슬라는 한달 전에 비해 주가가 올랐나 내렸나?
================================== Ai Message ==================================
Tool Calls:
  get_yf_stock_history (call_KcZw4j69FLVuVgK7e97ZOhkF)
 Call ID: call_KcZw4j69FLVuVgK7e97ZOhkF
  Args:
    stock_history_input: {'ticker': 'TSLA', 'period': '1mo'}
==========